In [1]:
import sympy as sp

In [243]:
x = sp.Symbol('x')
expression = 2*(x/2)**2
a = sp.diff(expression, x)
a

x

Metric

In [210]:
n = 4
coordinates = []
for i in range(n):
    coordinate = sp.Symbol(input(f'Enter the coordinate {i}'))
    coordinates.append(coordinate)
coordinates_sym = sp.Matrix([coordinates])
coordinates_sym

Enter the coordinate 0 t
Enter the coordinate 1 r
Enter the coordinate 2 \theta
Enter the coordinate 3 \phi


Matrix([[t, r, \theta, \phi]])

In [229]:
is_symmetrical = True
g = []
for i in range(n):
    g_row = []
    for j in range(n):
        g_ij = input(f'Enter the component g_{i}{j}: ')
        try:
            float(g_ij)
            is_number = True
        except:
            is_number = False
        if (is_number == False):
            g_ij = sp.Symbol(g_ij)
        g_row.append(g_ij)
    g.append(g_row)
g
g_sym = sp.Matrix(g)
g_sym

Enter the component g_00:  -f(r)
Enter the component g_01:  0
Enter the component g_02:  0
Enter the component g_03:  0
Enter the component g_10:  0
Enter the component g_11:  f(r)
Enter the component g_12:  0
Enter the component g_13:  0
Enter the component g_20:  0
Enter the component g_21:  0
Enter the component g_22:  r^2
Enter the component g_23:  0
Enter the component g_30:  0
Enter the component g_31:  0
Enter the component g_32:  0
Enter the component g_33:  r^{2}\sin^{2}{\theta}


Matrix([
[-f(r),    0,   0,                     0],
[    0, f(r),   0,                     0],
[    0,    0, r^2,                     0],
[    0,    0,   0, r^{2}\sin^{2}{\theta}]])

Computing of Christofell symbols

In [315]:
def christoffel_symbol(lmbd,mu,nu):
    index = mu
    g_component = lambda nu, sigma: sp.Symbol(f'g_{{{nu}{sigma}}}')
    g_component_up = lambda nu, sigma: sp.Symbol(f'g^{{{nu}{sigma}}}')
    christoffel_component_1 = lambda mu, nu: sp.Derivative(g_component(nu, sp.Symbol('\\sigma')), coordinates_sym[mu])
    christoffel_component_2 = lambda mu, nu: sp.Derivative(g_component(nu, sp.Symbol('\\sigma')), coordinates_sym[nu])
    christoffel_component_3 = lambda mu, nu: sp.Derivative(g_component(mu, nu), sp.Symbol('\\sigma'))
    return sp.Symbol(f'\\Gamma^{lmbd}_{{{mu}{nu}}} =  \\frac{1}{2}')*g_component_up(lmbd, sp.Symbol('\\sigma'))*(christoffel_component_1(mu,nu) + christoffel_component_2(mu,nu) - christoffel_component_3(mu,nu))
christoffel_symbol(1,2,3)

\Gamma^1_{23} =  \frac12*g^{1\sigma}*(-Derivative(g_{23}, \sigma) + Derivative(g_{3\sigma}, \phi) + Derivative(g_{3\sigma}, \theta))

In [ ]:
Hacer una para los cálculos ahora sí (este solo era visual)

In [260]:
christoffel_component = lambda mu, nu, sigma: sp.Derivative(sp.Symbol(f'({g_sym[nu + sigma]})'), coordinates_sym[mu])
christoffel_component = lambda mu, nu, sigma: sp.Derivative(sp.Symbol(f'g_{{{nu}{sigma}}}'), coordinates_sym[mu])
christoffel_component(1,3,0)

Derivative(g_{30}, r)

In [258]:
nu = 'μ'
sigma = 'σ'

# Esto crea: g_{μσ}
nombre = f"g_{{{nu}{sigma}}}"
simbolo = sp.Symbol(nombre)

simbolo  # g_{μσ}

g_{μσ}